In [1]:
def pressure(v, t, n=6.022e23):
    """Compute the pressure in pascals of an ideal gas.

        v -- volume of gas, in cubic meters
        t -- absolute temperature in degrees kelvin
        n -- particles of gas (default: one mole)
    """
    k = 1.38e-23  # Boltzmann's constant
    return n * k * t / v
help(pressure)
pressure(1, 273.15)

Help on function pressure in module __main__:

pressure(v, t, n=6.022e+23)
    Compute the pressure in pascals of an ideal gas.
    
    v -- volume of gas, in cubic meters
    t -- absolute temperature in degrees kelvin
    n -- particles of gas (default: one mole)



2269.974834

## 1.5 Control

In [3]:
def fib(n):
    """Compute the nth Fibonacci number, for n>=2."""
    pred, curr = 0, 1
    k = 2
    while k < n:
        pred, curr = curr, pred + curr
        k = k + 1
    return curr
result = fib(8)

In [4]:
assert fib(8) == 13, 'The 8th Fibonacci number should be 13'

Assertions. Programmers use assert statements to verify expectations, such as the output of a function being tested. An assert statement has an expression in a boolean context, followed by a quoted line of text (single or double quotes are both fine, but be consistent) that will be displayed if the expression evaluates to a false value.

In [8]:
 def fib_test():
        assert fib(2) == 1, 'The 2nd Fibonacci number should be 1'
        assert fib(3) == 1, 'The 3rd Fibonacci number should be 1'
        assert fib(50) == 7778742049, 'Error at the 50th Fibonacci number'
fib_test()

When writing Python in files, rather than directly into the interpreter, tests are typically written in the same file or a neighboring file with the suffix _test.py.

Doctests. Python provides a convenient method for placing simple tests directly in the docstring of a function.

In [11]:
def sum_naturals(n):
        """Return the sum of the first n natural numbers.

        >>> sum_naturals(10)
        55
        >>> sum_naturals(100)
        5050
        """
        total, k = 0, 1
        while k <= n:
            total, k = total + k, k + 1
        return total

In [13]:
from doctest import testmod
testmod()

TestResults(failed=0, attempted=2)

In [12]:
from doctest import run_docstring_examples
run_docstring_examples(sum_naturals, globals(), True)

Finding tests in NoName
Trying:
    sum_naturals(10)
Expecting:
    55
ok
Trying:
    sum_naturals(100)
Expecting:
    5050
ok


## 1.6 Higher-Order Functions

### 1.6.1 Functions as Arguments

In [14]:
def sum_naturals(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k, k + 1
    return total

def sum_cubes(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k*k*k, k + 1
    return total

def pi_sum(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + 8 / ((4*k-3) * (4*k-1)), k + 1
    return total



def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total

def cube(x):
    return x*x*x

def sum_cubes(n):
    return summation(n, cube)

sum_cubes(3)

36

### 1.6.2 Functions as General Methods

In [1]:
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess

def golden_update(guess):
    return 1/guess + 1

def square_close_to_successor(guess):
    return approx_eq(guess * guess, guess + 1)

def approx_eq(x, y, tolerance=1e-3):
    return abs(x - y) < tolerance

improve(golden_update, square_close_to_successor)

1.6176470588235294

In [7]:
from math import sqrt
phi = 1/2 + sqrt(5)/2
def improve_test():
    approx_phi = improve(golden_update, square_close_to_successor)
    assert approx_eq(phi, approx_phi), 'phi differs from its approximation'
    
improve_test()

### 1.6.3 Defining Functions III: Nested Definitions

In [8]:
def average(x, y):
    return (x + y)/2

def sqrt_update(x, a):
    return average(x, a/x)

def sqrt(a):
    def sqrt_update(x):
        return average(x, a/x)
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_update, sqrt_close)

### 1.6.4 Functions as Returned Values

In [10]:
def composel(f, g):
    def h(x):
        return f(g(x))
    return h


In [ ]:
def average(x, y):
    return (x + y)/2

def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess

def approx_eq(x, y, tolerance=1e-3):
    return abs(x - y) < tolerance

def sqrt(a):
    def sqrt_update(x):
        return average(x, a/x)
    def sqrt_close(x):
        return approx_eq(x * x, a)
    return improve(sqrt_update, sqrt_close)

result = sqrt(256)

### 1.6.5 Example: Newton's Method

In [12]:
def newton_update(f, df):
    def update(x):
        return x - f(x) / df(x)
    return update

def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)
    return improve(newton_update(f, df), near_zero)

def square_root_newton(a):
    def f(x):
        return x * x - a
    def df(x):
        return 2 * x
    return find_zero(f, df)

square_root_newton(64)

8.000001655289593

In [ ]:
def power(x, n):
    """Return x * x * x * ... * x for x repeated n times."""
    product, k = 1, 0
    while k < n:
        product, k = product * x, k + 1
    return product

def nth_root_of_a(n, a):
    def f(x):
        return power(x, n) - a
    def df(x):
        return n * power(x, n-1)
    return find_zero(f, df)

nth_root_of_a(3, 64)

### 1.6.6 Currying

We can use higher-order functions to convert a function that takes multiple arguments into a chain of functions that each take a single argumen. More specifically, given a function f(x, y), we can define a function g such that g(x)(y) is equivalent to f(x, y). Here, g is a higher-order function that takes in a single argument x and returns another function that takes in a single argument y. This transformation is called currying,

In [14]:
def curried_pow(x):
    def h(y):
        return pow(x,y)
    return h

curried_pow(2)(3)

8

In [15]:
def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start = start + 1
        
map_to_range(0, 10, curried_pow(2))

1
2
4
8
16
32
64
128
256
512


In [18]:
def curry2(f):
    """Return a curried version of the given two-argument function."""
    def g(x):
        def h(y):
            return f(x, y)
        return h
    return g

def uncurry2(g):
    """Return a two-argument version of the given curried function."""
    def f(x, y):
        return g(x)(y)
    return f

pow_curried = curry2(pow)
pow_curried(2)(5)

32

In [19]:
map_to_range(0, 10, pow_curried(2))

1
2
4
8
16
32
64
128
256
512


In [20]:
uncurry2(pow_curried)(2, 5)

32

### 1.6.7 Lambda Expressions

In Python, we can create function values on the fly using lambda expressions, which evaluate to unnamed functions. A lambda expression evaluates to a function that has a single return expression as its body. Assignment and control statements are not allowed.

In [22]:
def compose1(f, g):
    return lambda x: f(g(x))

s = lambda x: x * x
print(s)
s(12)

<function <lambda> at 0x00000247E85C1670>


144

In [32]:
def trace(fn):
    def wrapped(x):
        print('->',fn,'(',x,')')
        return fn(x)
    return wrapped

@trace
def triple(x):
    return 3 * x

triple(12)

-> <function triple at 0x00000247E85C1C10> ( 12 )


36

In this example, A higher-order function trace is defined, which returns a function that precedes a call to its argument with a print statement that outputs the argument. The def statement for triple has an annotation, @trace, which affects the execution rule for def. As usual, the function triple is created. However, the name triple is not bound to this function. Instead, the name triple is bound to the returned function value of calling trace on the newly defined triple function. In code, this decorator is equivalent to:

In [34]:
def triple(x):
    return 3 * x
triple = trace(triple)
print(triple)

<function trace.<locals>.wrapped at 0x00000247E6AB4CA0>


https://cs61a.org/articles/debugging/

## 1.7 Recursive Functions
We'll begin with an example problem: write a function that sums the digits of a natural number. When designing recursive functions, we look for ways in which a problem can be broken down into simpler problems. 

In this case, the operators % and // can be used to separate a number into two parts: its last digit and all but its last digit.

In [2]:
print(18117 % 10)
print(18117 // 10)

7
1811


In [3]:
def sum_digits(n):
        """Return the sum of the digits of positive integer n."""
        if n < 10:
            return n
        else:
            all_but_last, last = n // 10, n % 10
            return sum_digits(all_but_last) + last
sum_digits(18117)

18

### 1.7.1 The Anatomy of Recursive Functions

In [4]:
"""The iterative function constructs the result from the base 
case of 1 to the final total by successively multiplying in each term.  """ 
def fact_iter(n):
    total, k = 1, 1
    while k <= n:
        total, k = total * k, k + 1
    return total


"""The recursive function, on the other hand, 
constructs the result directly from the final term, n, 
and the result of the simpler problem, fact(n-1)."""
def fact(n):
    if n == 1:
        return 1
    else:
        return n * fact(n-1)

fact(4)

"""The functions fact_iter and fact also differ because 
the former must introduce two additional names, total and k, 
that are not required in the recursive implementation."""

24

While we can unwind the recursion using our model of computation, it is often clearer to think about recursive calls as functional abstractions. That is, we should not care about how fact(n-1) is implemented in the body of fact; we should simply trust that it computes the factorial of n-1. Treating a recursive call as a functional abstraction has been called a recursive leap of faith. 

We define a function in terms of itself, but simply trust that the simpler cases will work correctly when verifying the correctness of the function. In this example, we trust that fact(n-1) will correctly compute (n-1)!; we must only check that n! is computed correctly if this assumption holds. In this way, verifying the correctness of a recursive function is a form of proof by induction.

### 1.7.2 Mutual Recursion
When a recursive procedure is divided among two functions that call each other, the functions are said to be mutually recursive. As an example, consider the following definition of even and odd for non-negative integers:

- a number is even if it is one more than an odd number

- a number is odd if it is one more than an even number

- 0 is even

Using this definition, we can implement mutually recursive functions to determine whether a number is even or odd:

In [7]:
def is_even(n):
    if n == 0:
        return True
    else:
        return is_odd(n-1)

def is_odd(n):
    if n == 0:
        return False
    else:
        return is_even(n-1)

is_even(4)

True

Mutually recursive functions can be turned into a single recursive function by breaking the abstraction boundary between the two functions. In this example, the body of is_odd can be incorporated into that of is_even, making sure to replace n with n-1 in the body of is_odd to reflect the argument passed into it:

In [8]:
def is_even(n):
    if n == 0:
        return True
    else:
        if (n-1) == 0:
            return False
        else:
            return is_even((n-1)-1)

is_even(14)

True

### 1.7.3 Printing in Recursive Functions

In [9]:
def cascade(n):
    """Print a cascade of prefixes of n."""
    if n < 10:
        print(n)
    else:
        print(n)
        cascade(n//10)
        print(n)
        
cascade(2013)

2013
201
20
2
20
201
2013


As another example of mutual recursion, consider a two-player game in which there are n initial pebbles on a table. The players take turns, removing either one or two pebbles from the table, and the player who removes the final pebble wins. Suppose that Alice and Bob play this game, each using a simple strategy:

- Alice always removes a single pebble
- Bob removes two pebbles if an even number of pebbles is on the table, and one otherwise

Given n initial pebbles and Alice starting, who wins the game?

A natural decomposition of this problem is to encapsulate each strategy in its own function. This allows us to modify one strategy without affecting the other, maintaining the abstraction barrier between the two. In order to incorporate the turn-by-turn nature of the game, these two functions call each other at the end of each turn.

In [10]:
def play_alice(n):
    if n == 0:
        print("Bob wins!")
    else:
        play_bob(n-1)
        
def play_bob(n):
    if n == 0:
        print("Alice wins!")
    elif is_even(n):
        play_alice(n-2)
    else:
        play_alice(n-1)

play_alice(20)

Bob wins!


### 1.7.4 Tree Recursion

In [12]:
def fib(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    else:
        return fib(n-2) + fib(n-1) #先n-1再n-2也无妨
    

fib(6)

5

### 1.7.5 Example: Partitions

The number of partitions of a positive integer n, using parts up to size m, is the number of ways in which n can be expressed as the sum of positive integer parts up to m in increasing order. For example, the number of partitions of 6 using parts up to 4 is 9.

We will define a function count_partitions(n, m) that returns the number of different partitions of n using parts up to m. This function has a simple solution as a tree-recursive function, based on the following observation:

The number of ways to partition n using integers up to m equals

- the number of ways to partition n-m using integers up to m, and
- the number of ways to partition n using integers up to m-1.

To see why this is true, observe that all the ways of partitioning n can be divided into two groups: those that include at least one m and those that do not. Moreover, each partition in the first group is a partition of n-m, followed by m added at the end. In the example above, the first two partitions contain 4, and the rest do not.

Therefore, we can recursively reduce the problem of partitioning n using integers up to m into two simpler problems: (1) partition a smaller number n-m, and (2) partition with smaller components up to m-1.

In [13]:
def count_partitions(n,m):
    """Count the ways to partition n using parts up to m."""
    if n == 0:
        return 1
    elif n < 0:
        return 0
    elif m == 0:
        return 0
    else:
        return count_partitions(n-m, m) + count_partitions(n, m-1)

count_partitions(6, 4)

9

## Lab02

In [15]:
>>> def cake():
...    print('beets')
...    def pie():
...        print('sweets')
...        return 'cake'
...    return pie
>>> chocolate = cake()


>>> chocolate


>>> chocolate()


>>> more_chocolate, more_cake = chocolate(), cake


>>> more_chocolate


>>> def snake(x, y):
...    if cake == more_cake:
...        return chocolate
...    else:
...        return x + y
>>> snake(10, 20)

>>> snake(10, 20)()


>>> cake = 'cake'
>>> snake(10, 20)


beets
sweets
sweets
sweets


30

In [16]:
def count_factors(n):
    """Return the number of positive factors that n has.
    >>> count_factors(6)
    4   # 1, 2, 3, 6
    >>> count_factors(4)
    3   # 1, 2, 4
    """
    i = 1
    count = 0
    while i <= n:
        if n % i == 0:
            count += 1
        i += 1
    return count

def count_primes(n):
    """Return the number of prime numbers up to and including n.
    >>> count_primes(6)
    3   # 2, 3, 5
    >>> count_primes(13)
    6   # 2, 3, 5, 7, 11, 13
    """
    i = 1
    count = 0
    while i <= n:
        if is_prime(i):
            count += 1
        i += 1
    return count

def is_prime(n):
    return count_factors(n) == 2 # only factors are 1 and n